In [0]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from PIL import Image, ImageFile
import re
import nltk
import os
import pickle

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [0]:
#albums_path = '/Users/Nick/OneDrive/College/2018 Fall/Data Science Lab EE460J/Final Project/output'
#output_path = '/Users/Nick/OneDrive/College/2018 Fall/Data Science Lab EE460J/Final Project/embeddings'

albums_path = '/content/drive/Team Drives/Audio Vision/Models/output'
output_path = '/content/drive/Team Drives/Audio Vision/Models/embeddings'

In [0]:
def format_data(albums_path):
    all_lyrics = []
    
    for dirName, subdirList, fileList in os.walk(albums_path, topdown=False):
        for fname in fileList:
            
            if fname.endswith('.txt'):
                lyrics_file = open(dirName + '/' + fname, "r")
                lyrics = lyrics_file.read()
                lyrics_file.close()
                
                
                lyrics = lyrics.replace('\n', ' ')
                lyrics = lyrics.replace('\\', '')
                lyrics = lyrics.strip(' ')
                #lyrics = lyrics.replace(r"\[.*\]","")
                lyrics = re.sub(r'\[[^\]]*\]', '', lyrics)
                all_lyrics.append(lyrics.lower())
                
                
            if fname.endswith('.jpg'):
                #Hidden file issues in colab
                resized_img = Image.open(dirName + '/' + fname).resize((256, 256))
                resized_img.save(dirName + '/' + fname)
            
    return all_lyrics

In [0]:
lyrics = format_data(albums_path)

tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(lyrics)]

In [0]:
max_epochs = 100
vec_size = 256
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm=1,
                iter=5)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

In [0]:
# from gensim.models.doc2vec import Doc2Vec

# model= Doc2Vec.load("d2v.model")

# #to find the vector of a document which is not in training data
# test_data = word_tokenize("Smells like teen spirit".lower())
# v1 = model.infer_vector(test_data)
# print("V1_infer", v1)

# # to find most similar doc using tags
# similar_doc = model.docvecs.most_similar('1')
# print(similar_doc)


# # to find vector of doc in training data using tags or in other words, printing the vector of document at index 1 in training data
# print(model.docvecs['1'])

In [0]:
def output_embeddings(output_path, albums_path):
    embeddings = {}
    
    for dirName, subdirList, fileList in os.walk(albums_path, topdown=False):
        album_lyrics = ''
        album_name = dirName.split('/')[-1]
        
        for fname in fileList:
            
            if fname.endswith('.txt'):
                lyrics_file = open(dirName + '/' + fname, "r")
                lyrics = lyrics_file.read()
                lyrics_file.close()
                
                
                lyrics = lyrics.replace('\n', ' ')
                lyrics = lyrics.replace('\\', '')
                lyrics = lyrics.strip(' ')
                lyrics = re.sub(r'\[[^\]]*\]', '', lyrics)
                album_lyrics = album_lyrics + lyrics.lower() + ' '
        
        if not len(album_lyrics) == 0:
            temp = word_tokenize(album_lyrics)
            vector = model.infer_vector(temp)
            embeddings[album_name.replace('/','-')] = vector
    
    return embeddings

In [0]:
embeddings = output_embeddings(output_path, albums_path)

In [0]:
file = "embeddings"

file_object = open(file,'wb')

pickle.dump(embeddings, file_object)

file_object.close()